In [5]:
from src.preprocessing import load_images_from_folder, augment_images
from src.model import build_model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np

# 1. Carrega imagens
X, y = load_images_from_folder("data/lavouras")
X_aug, y_aug = augment_images(X, y)

# 2. Codifica labels
le = LabelEncoder()
y_all = le.fit_transform(np.concatenate([y, y_aug]))
X_all = np.concatenate([X, X_aug])

# 3. Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

# 4. Modelo
model = build_model(num_classes=len(le.classes_))
model.fit(X_train / 255.0, y_train, validation_data=(X_test / 255.0, y_test), epochs=10)

# 5. Avaliação
y_pred = model.predict(X_test / 255.0).argmax(axis=1)
print(classification_report(y_test, y_pred, target_names=le.classes_))


ModuleNotFoundError: No module named 'src'